In [1]:
import pandas as pd
import numpy as np
import joblib
import os
import requests
from io import StringIO
from datetime import datetime
from google.colab import files

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Script para usar o modelo de IA treinado e fazer novas previsões com dados reais
# Responsável: Gustavo


# --- CONFIGURAÇÕES E DADOS DE REFERÊNCIA ---
CENTROS_HOTSPOTS = {
    "Farallon_Islands": (37.7, -123.0),
    "Ano_Nuevo": (37.1, -122.3),
    "Point_Reyes": (38.0, -123.0)
}
NOME_FICHEIRO_MODELO = "modelo_tubarao_v1.pkl"

def buscar_dado_real_erddap(dataset_id, variavel, lat, lon):
    """
    Busca um dado real e recente de um dataset no ERDDAP.
    """
    base_url = f"https://coastwatch.pfeg.noaa.gov/erddap/griddap/{dataset_id}"

    # Pede o dado para o momento mais recente disponível
    query_url = (
        f"{base_url}.csv?"
        f"{variavel}"
        f"[last][({lat})][({lon})]"
    )

    try:
        response = requests.get(query_url, timeout=30)
        response.raise_for_status()
        dados_csv = pd.read_csv(StringIO(response.text), skiprows=[1])
        valor = dados_csv[variavel].iloc[0]

        # Converte de Kelvin para Celsius se a variável for 'analysed_sst'
        if variavel == 'analysed_sst' and valor is not None:
            return valor - 273.15

        return valor
    except Exception as e:
        print(f"  -> AVISO: Falha ao buscar '{variavel}'. Será usado um valor simulado. (Erro: {e})")
        return None # Retorna None em caso de falha

def buscar_dados_reais_oceanograficos(lat, lon):
    """
    Busca os dados reais de SST e Clorofila via API (ERDDAP).
    """
    print("\n1. A buscar dados oceanográficos reais via API...")

    # Tenta buscar a Temperatura (SST) do dataset Nível 4 (mais confiável)
    sst_real = buscar_dado_real_erddap('MUR-JPL-L4-GLOB-v4.1', 'analysed_sst', lat, lon)

    # Tenta buscar a Clorofila
    clorofila_real = buscar_dado_real_erddap('VIIRS_NPP_L3M_KD490_DAILY_4KM_R2022', 'chlor_a', lat, lon)

    return sst_real, clorofila_real

def calcular_proximidade_colonia(lat, lon):
    # ... (código da função sem alterações)
    distancias = []
    for nome, (lat_centro, lon_centro) in CENTROS_HOTSPOTS.items():
        dist = np.sqrt((lat - lat_centro)**2 + (lon - lon_centro)**2) * 111
        distancias.append(dist)
    return min(distancias)

def simular_parametros_restantes(lat, lon, sst_fallback, clorofila_fallback):
    """
    Simula os parâmetros que não foram obtidos via API.
    Usa os valores reais como base para a simulação, se disponíveis.
    """
    np.random.seed(int(abs(lat) * 100 + abs(lon) * 100))

    # Usa o valor real se existir, senão simula
    sst = sst_fallback if sst_fallback is not None else 14 + (38 - lat) + np.random.uniform(-1.5, 1.5)
    clorofila = clorofila_fallback if clorofila_fallback is not None else np.random.uniform(0.2, 4.0)

    # Simula os parâmetros restantes
    profundidade = np.random.uniform(50, 400)
    declive = np.random.uniform(5, 40)
    energia = np.random.uniform(0.3, 0.9)

    return sst, profundidade, declive, clorofila, energia

def fazer_previsao(latitude, longitude):
    """
    Função principal que busca dados reais, simula o restante
    e usa o modelo treinado para calcular a probabilidade.
    """
    # --- 1. Buscar e Preparar as Features ---
    sst_real, clorofila_real = buscar_dados_reais_oceanograficos(latitude, longitude)

    proximidade = calcular_proximidade_colonia(latitude, longitude)
    sst, profundidade, declive, clorofila, energia = simular_parametros_restantes(latitude, longitude, sst_real, clorofila_real)

    novo_local = pd.DataFrame([{
        'sst_c': sst,
        'proximidade_colonia_km': proximidade,
        'profundidade_m': profundidade,
        'declive_graus': declive,
        'clorofila_mg_m3': clorofila,
        'energia_dinamica_score': energia
    }])

    print("\n   - Dados de entrada para o modelo (combinando real e simulado):")
    print(novo_local.iloc[0].to_string())

    # --- 2. Carregar o modelo de IA ---
    print(f"\n2. A carregar o modelo de IA do ficheiro '{NOME_FICHEIRO_MODELO}'...")
    try:
        modelo_carregado = joblib.load(NOME_FICHEIRO_MODELO)
    except FileNotFoundError:
        print(f"\n--- ERRO ---")
        print(f"O ficheiro do modelo '{NOME_FICHEIRO_MODELO}' não foi encontrado.")
        return

    # --- 3. Fazer a previsão ---
    print("\n3. A calcular a probabilidade...")
    probabilidade = modelo_carregado.predict_proba(novo_local)[:, 1][0]

    # --- 4. Apresentar o resultado ---
    print("\n--- RESULTADO DA PREVISÃO ---")
    print(f"Para as coordenadas ({latitude}, {longitude}), a probabilidade de avistamento de um tubarão-branco é de: {probabilidade * 100:.2f}%")

# --- Ponto de Entrada do Script ---
if __name__ == "__main__":
    print("--- Ferramenta de Previsão de Hotspots de Tubarões-Brancos (com Dados Reais) ---")

    try:
        lat_input = float(input("Introduza a Latitude (ex: 37.7): "))
        lon_input = float(input("Introduza a Longitude (ex: -123.0): "))

        fazer_previsao(lat_input, lon_input)

    except ValueError:
        print("\nErro: Por favor, introduza valores numéricos válidos para as coordenadas.")
    except Exception as e:
        print(f"\nOcorreu um erro inesperado: {e}")

--- Ferramenta de Previsão de Hotspots de Tubarões-Brancos (com Dados Reais) ---
Introduza a Latitude (ex: 37.7): 37.7
Introduza a Longitude (ex: -123.0): -123.0

1. A buscar dados oceanográficos reais via API...
  -> AVISO: Falha ao buscar 'analysed_sst'. Será usado um valor simulado. (Erro: 404 Client Error:  for url: https://coastwatch.pfeg.noaa.gov/erddap/griddap/MUR-JPL-L4-GLOB-v4.1.csv?analysed_sst%5Blast%5D%5B(37.7)%5D%5B(-123.0)%5D)
  -> AVISO: Falha ao buscar 'chlor_a'. Será usado um valor simulado. (Erro: 404 Client Error:  for url: https://coastwatch.pfeg.noaa.gov/erddap/griddap/VIIRS_NPP_L3M_KD490_DAILY_4KM_R2022.csv?chlor_a%5Blast%5D%5B(37.7)%5D%5B(-123.0)%5D)

   - Dados de entrada para o modelo (combinando real e simulado):
sst_c                      15.485036
proximidade_colonia_km      0.000000
profundidade_m            378.601043
declive_graus              30.723165
clorofila_mg_m3             1.506567
energia_dinamica_score      0.368373

2. A carregar o modelo de IA